In [ ]:
import os
import boto3
import time
import numpy as np
import sagemaker

sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket_name = "deependu-my-personal-projects"
jobs_folder = "jobs"
dataset_folder = "datasets"
project_directory = "image-captioning-project"  # 'dummy-image-captioning-dataset'

In [ ]:
current_time = time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())
job_name = f'pytorch-smddp-dist-image-captioning-{current_time}'
output_path = f"s3://{bucket_name}/{project_directory}/{jobs_folder}"

hyperparameters = {
    "epochs": 2,
    "lr": 0.01,
    "batch-size": 256,
    "backend": "smddp",
}

In [ ]:
distribution = {"smdistributed": {"dataparallel": {"enabled": True}}}

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="training_script.py",
    source_dir="code",
    output_path=output_path + "/",
    code_location=output_path,
    role=role,
    instance_count=2,
    instance_type="ml.p4d.24xlarge",  # 'ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge',
    py_version="py310",
    framework_version="2.0.1",
    distribution=distribution,
    hyperparameters=hyperparameters,
)

In [ ]:
datasets = (
    f"s3://{bucket_name}/{project_directory}/{dataset_folder}"  # s3 location of dataset
)
datasets

In [ ]:
estimator.fit({"train": datasets}, job_name=job_name, wait=True)